# EAMCET Zero Manual Pipeline - Google Colab

This notebook runs the fully automated EAMCET AI tutor training pipeline without any manual annotations required.

## Features:
- ✅ No manual annotations needed
- ✅ Intelligent pattern recognition
- ✅ Automatic question and answer extraction
- ✅ Automated model training
- ✅ Works with any EAMCET PDF format

## Step 1: Setup Environment

Install all dependencies and clone the repository.

In [ ]:
# Install system dependencies
!apt-get install -y tesseract-ocr tesseract-ocr-eng libtesseract-dev libgl1-mesa-glx libglib2.0-0

# Clone repository (UPDATE THIS URL WITH YOUR GITHUB REPOSITORY)
!git clone https://github.com/YOUR_USERNAME/eamcet_ai_tutor.git

# Install Python dependencies
%cd eamcet_ai_tutor
!pip install -r requirements.txt

# Verify installation
import fitz
print("✅ PyMuPDF imported successfully")
print("✅ Environment ready!")

## Step 2: Upload Your EAMCET PDFs

Upload your EAMCET PDF files to the data/raw_pdfs folder.

In [ ]:
from google.colab import files
import os
from pathlib import Path

# Create data directory structure
os.makedirs("data/raw_pdfs", exist_ok=True)

# Upload files
uploaded = files.upload()

# Move uploaded files to data folder
for filename in uploaded.keys():
    if filename.endswith('.pdf'):
        os.rename(filename, f"data/raw_pdfs/{filename}")
        print(f"✅ Moved {filename} to data/raw_pdfs/")

print(f"\n📁 Total PDFs uploaded: {len([f for f in os.listdir('data/raw_pdfs') if f.endswith('.pdf')])}")

## Step 3: Run Zero Manual Pipeline

Execute the fully automated extraction and training pipeline.

In [ ]:
# Run the zero manual pipeline
!python eamcet_zero_manual_pipeline.py --input_folder data/raw_pdfs --output_folder colab_results

print("\n✅ Pipeline completed!")
print("📁 Check the colab_results folder for outputs")

## Step 4: View Results

Examine the extracted data and training results.

In [ ]:
import json
import pandas as pd
from pathlib import Path

# Load pipeline summary
if Path("colab_results/pipeline_summary.json").exists():
    with open("colab_results/pipeline_summary.json", "r") as f:
        summary = json.load(f)
    
    print("📊 PIPELINE SUMMARY:")
    print("=" * 40)
    
    print(f"Total questions extracted: {summary['extraction_stats']['total_questions']}")
    print(f"Questions with answers: {summary['extraction_stats']['paired_questions']}")
    
    print("\n📚 Subject Breakdown:")
    for subject, count in summary['extraction_stats']['subjects'].items():
        if count > 0:
            print(f"  {subject}: {count} questions")
    
    print("\n🎯 Training Data Created:")
    for data_type, count in summary['training_data_stats'].items():
        print(f"  {data_type}: {count} samples")
else:
    print("❌ Pipeline summary not found. Check if pipeline completed successfully.")

## Step 5: Download Results

Download the processed data and trained models.

In [ ]:
from google.colab import files
import zipfile
import os

# Create zip file of results
if os.path.exists("colab_results"):
    with zipfile.ZipFile("eamcet_results.zip", "w") as zipf:
        for root, dirs, files in os.walk("colab_results"):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, "colab_results")
                zipf.write(file_path, arcname)
    
    # Download the zip file
    files.download("eamcet_results.zip")
    print("✅ Results downloaded as eamcet_results.zip")
else:
    print("❌ No results found to download")